In [1]:
import sys; sys.path.insert(0, '..')
from database.dataset import *

from feature_extraction import CandidatesFeatureExtraction, CandidatesFeatureExtractionMP


In [4]:
db = INBreast_Dataset(
        return_lesions_mask=True,
        level='image',
        max_lesion_diam_mm=1.0,
        extract_patches=True,
        extract_patches_method='all',  # 'centered'
        patch_size=256,
        stride=256,
        min_breast_fraction_roi=0.5,
        normalize=None,
        n_jobs=-1,
)

In [5]:
idx = 141
image = db[idx]['img']
image_id = db.df.iloc[idx].img_id
image_mask = db[idx]['lesion_mask']

Available feature groups are: `['firstorder', 'glcm', 'gldm', 'glrlm', 'glszm', 'ngtdm', 'shape', 'shape2D']`.


To include all features for a feature group use `[]` or `None` as the key in settings dict, like: `{'firstorder':[]}`, otherwise we can specify features using their names `{'firstorder':['10Percentile]}`

Image types specify for which images should these features be calculated for: original image, normalized, wavelet, etc.

https://pyradiomics.readthedocs.io/en/latest/customization.html


In [6]:
feature_types = {'firstorder':[], 'shape2D':[]}
image_types = {'Original':{}}
candidates = np.array([[300, 300, 10]]*1000)


# FE class initialization
cfe  = CandidatesFeatureExtraction(feature_types, image_types, patch_size=30)
cfe_mp  = CandidatesFeatureExtractionMP(feature_types, image_types, patch_size=30, n_jobs=6)

In [7]:
candidates = np.array([[300, 300, 10]]*1000)

In [8]:
res_mp = cfe_mp.extract_features(candidates, image, image_mask, sample=100)

In [9]:
res = cfe.extract_features(candidates, image, image_mask, sample=100)

100%|██████████| 100/100 [00:23<00:00,  4.17it/s]


In [15]:
# check that MP and regular classes return the same
assert np.all((pd.DataFrame(res) == pd.DataFrame(res_mp)).values)

In [16]:
pd.DataFrame(res)

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance,coordinates,patch_mask_intersection
0,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
1,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
2,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
3,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
4,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
96,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
97,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
98,v3.0.1,1.20.3,2.1.1,1.1.1,3.9.7,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},d06edffafa494a123773ee22c0058e0c78f6c0d3,2D,"(1.0, 1.0)",...,1006.0,282.0,30.392436365838012,1147.319416436998,-0.05816515036088505,1184707659.0,0.13043456790123456,2967.26934691358,"((285, 315), (285, 315))",0
